# MiniMax algorithm in Reversi game

In [113]:
import numpy as np
from copy import deepcopy
import time

## Generating game states and possible moves

In [138]:
def PossibleMoves(player, game_array):

    available_moves=[]

    for row in range(8):
        for col in range(8):
            if game_array[row][col]==0:
                for row_direct,col_direct in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]:

                    i=1
                    found = False
                    while (row + row_direct*i < 8) and (row + row_direct*i) >=0 and (col + col_direct*i) < 8 and (col + col_direct*i) >=0:

                        if game_array[row + row_direct*i][col + col_direct*i]==0:
                            break

                        if game_array[row + row_direct*i][col + col_direct*i]==player:
                            if i!=1:
                                available_moves.append((row,col))
                                found = True
                            break
                        
                        i+=1

                    if found==True:
                        break   

    return available_moves                

In [139]:
def PossibleBoards(player, game_array):
    
    possible_boards={}
    moves_array = PossibleMoves(player, game_array)

    for row,col in moves_array:

        new_board = deepcopy(game_array)

        for row_direct,col_direct in [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]:

            i=1
            
            while (row + row_direct*i < 8) and (row + row_direct*i) >=0 and (col + col_direct*i) < 8 and (col + col_direct*i) >=0:

                if game_array[row + row_direct*i][col + col_direct*i]==0:
                    break

                if game_array[row + row_direct*i][col + col_direct*i]==player:
                    if i!=1:
                        for j in range(i):
                            new_board[row + row_direct*j][col + col_direct*j]=player
                        break
                        
                i+=1
                
        possible_boards[(row,col)]=new_board

    return possible_boards

In [140]:
exampl_board= np.array([
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 2, 0, 0, 0],
       [0, 0, 1, 1, 2, 2, 0, 0],
       [0, 1, 1, 2, 0, 2, 0, 0],
       [0, 0, 2, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])
    

In [141]:
PossibleMoves(2, exampl_board)

[(1, 2), (1, 3), (2, 4), (3, 1), (3, 2), (4, 0), (4, 1), (5, 0), (6, 1)]

In [142]:
PossibleBoards(2, exampl_board)

{(1,
  2): array([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 0, 0],
        [0, 0, 2, 2, 0, 0, 0, 0],
        [0, 0, 0, 1, 2, 0, 0, 0],
        [0, 0, 1, 1, 2, 2, 0, 0],
        [0, 1, 1, 2, 0, 2, 0, 0],
        [0, 0, 2, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]),
 (1,
  3): array([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 0, 0, 0, 0],
        [0, 0, 2, 2, 0, 0, 0, 0],
        [0, 0, 0, 2, 2, 0, 0, 0],
        [0, 0, 1, 2, 2, 2, 0, 0],
        [0, 1, 1, 2, 0, 2, 0, 0],
        [0, 0, 2, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]),
 (2,
  4): array([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 2, 2, 2, 0, 0, 0],
        [0, 0, 0, 1, 2, 0, 0, 0],
        [0, 0, 1, 1, 2, 2, 0, 0],
        [0, 1, 1, 2, 0, 2, 0, 0],
        [0, 0, 2, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]),
 (3,
  1): array([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 2, 1, 0, 0, 0, 0],
        [0, 2, 0, 1, 2, 0, 0, 0],
  

# Strategies

### Possible strategies:

* higher number of flipped disk is better
* at the beginning the least the better (up to using half of your discs)
* place pieces strategically around the board to avoid getting boxed in
* wait to place disks in spaces where your opponent can’t play
* avoid playing disks in the spaces immediately next to the corners or next to the edge rows
* leaving your opponent with as few legal moves as possible while not limiting your own choices
* leaving your opponent with no legal moves forceing him to forfeit his turn
* forceing your opponent into making a bad move

In [82]:
#random move for comparison of algorithms effectivness
import random
def RandomMove(moves_dict):
    return random.choice(list(moves_dict.values()))

In [143]:
def ChooseMoveWithMostDiscs(moves_dict, player):
    max_disks = 0
    for board in list(moves_dict.values()):
        disc_count = np.count_nonzero(board == player)
        if disc_count > max_disks:
            max_disks = disc_count
            best_board = board

    return best_board

In [144]:
def CountPlayerDisks(board, player):
    return np.count_nonzero(board == player)

In [145]:
STABLE_POS_VALUE = 1

def sum_ar(n):
    if n==0:
        return 0   
    return n + sum_ar(n-1)


def StablePositions(board, player):

    moving_dir = ((1,1), (1,-1),(-1,1),(-1,-1))
    quality = 0
    
    for i_dir, j_dir in moving_dir:
    
        adj_wall_len = 9
        
        i_corner = 0 if i_dir == 1 else 7
        j_corner = 0 if j_dir == 1 else 7

        i_opposite_breakpoint = 8 if i_dir == 1 else -1
        j_opposite_breakpoint = 8 if j_dir == 1 else -1

        i = i_corner 
        j = j_corner

        while True:

            if j==j_opposite_breakpoint: break #reached end of board
                    
            if board[i][j]==player and (abs(i_corner-i)<adj_wall_len-1): 
                quality+=STABLE_POS_VALUE
                i+=1*i_dir
                if i == i_opposite_breakpoint:
                    i = i_corner
                    j +=1*j_dir
            else:
                if i == i_corner:
                    break
                else:
                    adj_wall_len = abs(i_corner-i)
                    i=i_corner
                    j+=1*j_dir
        
        corner = sum_ar(abs(j_corner-j))
        

        adj_wall_len = 9

        temp = i_dir
        i_dir = j_dir
        j_dir = temp

        i_corner = 0 if i_dir == 1 else 7
        j_corner = 0 if j_dir == 1 else 7

        i_opposite_breakpoint = 8 if i_dir == 1 else -1
        j_opposite_breakpoint = 8 if j_dir == 1 else -1

        i = i_corner 
        j = j_corner

        while True:

            if j==j_opposite_breakpoint: break #reached end of board
                    
            if board[j][i]==player and (abs(i_corner-i)<adj_wall_len-1): 
                quality+=STABLE_POS_VALUE
                i+=1*i_dir
                if i == i_opposite_breakpoint:
                    i = i_corner
                    j +=1*j_dir
            else:
                if i == i_corner:
                    break
                else:
                    adj_wall_len = abs(i_corner-i)
                    i=i_corner
                    j+=1*j_dir

        
        quality-=corner
                
    return quality 
 

In [146]:
def CornerEdges(board, player):
    cost = 0
    c_points = [(0,1), (1,0), (6,0), (7,1), (0,6), (1,7), (6,7), (7,6)]
    x_points = [(1,1), (1,6), (6,1), (6,6)]
    
    if board[0][0]==0:
        if board[0][1]==player:
            cost+=1
        if board[1][0]==player:
            cost+=1    
        if board[1][1]==player:
            cost+=2

    if board[0][7]==0:
        if board[0][6]==player:
            cost+=1
        if board[1][7]==player:
            cost+=1    
        if board[1][6]==player:
            cost+=2   

    if board[7][0]==0:
        if board[6][0]==player:
            cost+=1
        if board[7][1]==player:
            cost+=1    
        if board[6][1]==player:
            cost+=2        
    
    if board[7][7]==0:
        if board[6][7]==player:
            cost+=1
        if board[7][6]==player:
            cost+=1    
        if board[6][6]==player:
            cost+=2 

    return cost

In [147]:
def SafeSpots(board, player): # spots where opponent can't play 

    opponent_moves = PossibleMoves(1 if player==2 else 2, board)
    player_moves = PossibleMoves(player, board)

    count = 0
    
    for move in player_moves:
        if move not in opponent_moves:
            count+=1
    return count

In [148]:
def NumberOfMoves(board, player):
    return len(PossibleMoves(player, board))

In [149]:
ex2= np.array([
       [2, 2, 1, 0, 1, 0, 2, 0],
       [2, 1, 1, 2, 1, 0, 0, 1],
       [1, 1, 0, 1, 0, 0, 0, 1],
       [1, 1, 0, 1, 2, 0, 0, 0],
       [1, 0, 0, 2, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [1, 1, 0, 0, 0, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 1]])

print(f"First player disks: {CountPlayerDisks(ex2, 1)}")
print(f"Number of possible moves: {NumberOfMoves(ex2, 1)}")
print(f"Number of safe spots: {SafeSpots(ex2, 1)}")
print(f"Dangerous spots aroud corners (x point costs twice): {CornerEdges(ex2, 1)}")
print(f"Dangerous spots aroud corners forced on opponent: {CornerEdges(ex2, 2)}")
print(f"Stable positions: {StablePositions(ex2, 1)}")

First player disks: 28
Number of possible moves: 7
Number of safe spots: 4
Dangerous spots aroud corners (x point costs twice): 1
Dangerous spots aroud corners forced on opponent: 1
Stable positions: 12


In [150]:

def HeuristicFunction(board, version):

    heuristic_fuctions = [StablePositions, CountPlayerDisks, SafeSpots] if version=="v1" else [StablePositions, NumberOfMoves, CornerEdges]

    weights = [-0.5, 0.5, 0.3] if version=="v1" else [0.5, 0.5, 0.3]
    sides = [2,1,1] if version=="v1" else [1, 1, 1]
    quality = 0
    
    for heur_fun, weight,side in zip(heuristic_fuctions, weights, sides):
           
        quality += weight * heur_fun(board, side)

    return quality

# MiniMax

In [132]:
CUTTINGPOINT = 4

def MinMax(player, board, i, version):

    children_dict_player = PossibleBoards(player, board)
    opponent = 1 if player==2 else 2

    if not bool(children_dict_player) and not bool(PossibleBoards(opponent, board)):
        return 64*10 if np.count_nonzero(board == 1)>np.count_nonzero(board == 2) else -64*10

    if i == CUTTINGPOINT:
        return HeuristicFunction(board, version)
    
    if player==1:
        if not bool(children_dict_player):
            return MinMax(opponent, board, i+1, version)
        return max([MinMax(opponent, child_board, i+1, version) for child_board in children_dict_player.values()])
    else:
        if not bool(children_dict_player):
            return MinMax(opponent, board, i+1, version)
        return min([MinMax(opponent, child_board, i+1, version) for child_board in children_dict_player.values()])  

In [155]:
def OthelloSimulation():
   
   player1 = 1
   player2 = 2

   starting_board = np.array(
      [[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 2, 0, 0, 0],
       [0, 0, 0, 2, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])
   
   current_board = starting_board
   j=0
   while True:
      print(f"Both players loop number: {j}")
      
      moves_dict_player1 = PossibleBoards(player1, current_board)
      if bool(moves_dict_player1):
         results = {key: MinMax(2, board, 1, "v1") for key,board in moves_dict_player1.items()}
         current_board = moves_dict_player1[max(results, key=results.get)]
         print(current_board)
      moves_dict_player2 = PossibleBoards(player2, current_board) 

      if bool(moves_dict_player2):
         results = {key: MinMax(1, board, 1, "v2") for key,board in moves_dict_player2.items()}
         current_board = moves_dict_player2[min(results, key=results.get)]
         print(current_board)
         
      if not bool(moves_dict_player1) and not bool(moves_dict_player2):  
         break
      j=j+1
   if np.count_nonzero(current_board == 1)>np.count_nonzero(current_board == 2):
      print("Player 1 wins!")
   else:
      print("Player 2 wins!")
   return current_board, np.count_nonzero(current_board == 1), np.count_nonzero(current_board == 2) 

In [156]:
#final_board, player1_count, player2_count = OthelloSimulation()
#print(final_board)

p1_wins=0
p2_wins=0
for i in range(1):
    _, player1_count, player2_count = OthelloSimulation()
    if player1_count>player2_count:
        p1_wins+=1
    elif  player1_count<player2_count:
        p2_wins+=1
print(p1_wins,p2_wins)    

Both players loop number: 0
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 2 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 2 2 2 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
Both players loop number: 1
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 1 2 2 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 1 2 0 0 0]
 [0 0 0 1 2 2 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
Both players loop number: 2
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 1 2 0 0 0]
 [0 0 0 1 1 1 1 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 1 2 0 0 0]
 [0 0 0 1 1 2 1 0]
 [0 0 0 1 0 0 2 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]

##  Alpha Beta

In [135]:
CUTTINGPOINT = 4

def MinMaxAlphaBeta(player, board, i, alpha, beta, version):

    children_dict_player = PossibleBoards(player, board)
    opponent = 1 if player==2 else 2

    if not bool(children_dict_player) and not bool(PossibleBoards(opponent, board)):
        return 64*10 if np.count_nonzero(board == 1)>np.count_nonzero(board == 2) else 0

    if i == CUTTINGPOINT:
        return HeuristicFunction(board, version)
    
    if player==1:
        if not bool(children_dict_player):
            return MinMaxAlphaBeta(opponent, board, i+1,  alpha, beta, version)
        else:
            for child_board in children_dict_player.values():
                score = MinMaxAlphaBeta(opponent, child_board, i+1, alpha, beta, version)
                if score > alpha:
                   alpha = score 
                if alpha >= beta:
                    return alpha
            return alpha    
    else:
        if not bool(children_dict_player):
            return MinMaxAlphaBeta(opponent, board, i+1, alpha, beta, version)
        else:
            for child_board in children_dict_player.values():
                score = MinMaxAlphaBeta(opponent, child_board, i+1, alpha, beta, version)
                if score < beta:
                   beta = score 
                if alpha >= beta:
                    return beta
            return beta  

In [136]:
def OthelloSimulationAlphaBeta(starting_player=1):
   
   player1 = starting_player
   player2 = 1 if starting_player==2 else 2

   starting_board = np.array(
      [[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 2, 0, 0, 0],
       [0, 0, 0, 2, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])
   
   current_board = starting_board
   j=0
   while True:

      print(j)
      moves_dict_player1 = PossibleBoards(player1, current_board)
      if bool(moves_dict_player1):
         
         alpha = -float('inf')
         beta = float('inf')
         
         for child_board in moves_dict_player1.values():
            score = MinMaxAlphaBeta(player2, child_board, 1, alpha, beta, "v1")
            if score > alpha:
               alpha = score 
               best_child = child_board

         current_board = best_child
         print(current_board)
         

      moves_dict_player2 = PossibleBoards(player2, current_board) 
      if bool(moves_dict_player2):
         alpha = -float('inf')
         beta = float('inf')
         
         for child_board in moves_dict_player2.values():
            score = MinMaxAlphaBeta(player1, child_board, 1, alpha, beta, "v2")
            if score < beta:
               beta = score 
               best_child = child_board

         current_board = best_child
         print(current_board)
      if not bool(moves_dict_player1) and not bool(moves_dict_player2):  
         break
      j=j+1
   return current_board, np.count_nonzero(current_board == 1), np.count_nonzero(current_board == 2) 


In [137]:
p1_wins=0
p2_wins=0
for i in range(1):
    _, player1_count, player2_count = OthelloSimulationAlphaBeta()
    if player1_count>player2_count:
        p1_wins+=1
    elif  player1_count<player2_count:
        p2_wins+=1
print(p1_wins,p2_wins)  

0
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 2 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 2 2 2 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
1
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 1 2 2 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 1 2 0 0 0]
 [0 0 0 1 2 2 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
2
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 1 2 0 0 0]
 [0 0 0 1 1 1 1 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 1 2 0 0 0]
 [0 0 0 1 1 2 1 0]
 [0 0 0 1 0 0 2 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
3
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0

In [120]:
p1_wins, p2_wins, times = 0, 0, 0
for i in range(5):
    start = time.time()
    _, player1_count, player2_count = OthelloSimulationAlphaBeta()
    end = time.time()
    times+=end-start
    if player1_count>player2_count:
        p1_wins+=1
    elif  player1_count<player2_count:
        p2_wins+=1

print(p1_wins,p2_wins)   
print(times/5)

0 5
7.0185966968536375


In [121]:
p1_wins, p2_wins, times = 0, 0, 0
for i in range(5):
    start = time.time()
    _, player1_count, player2_count = OthelloSimulation()
    end = time.time()
    times+=end-start
    if player1_count>player2_count:
        p1_wins+=1
    elif  player1_count<player2_count:
        p2_wins+=1
print(p1_wins,p2_wins)     
print(times/5)

0 5
31.577995014190673
